# Debuggin della funzione 'Series to Supervised'

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import concat

In [2]:
#Lettura del File
df=pd.read_excel('SHILLER_Data.xlsx')

#Impostazione della data come indice
df.set_index('Date', inplace=True)
df.head()

#Eliminazione delle colonne Dividends e CPI
df=df.drop(['Dividends' ,'CPI'],axis=1)
df.dropna()

,S&P 500,Earnings,10Y Usa Rate
Date,,,
1900-01-31,6.100000,0.480000,3.150000
1900-02-28,6.210000,0.480000,3.145833
1900-03-31,6.260000,0.480000,3.141667
1900-04-30,6.340000,0.480000,3.137500
1900-05-31,6.040000,0.480000,3.133333
...,...,...,...
2023-08-31,4457.358696,183.170000,4.170000
2023-09-30,4409.095000,184.250000,4.380000
2023-10-31,4269.400909,186.976667,4.800000


In [3]:
# Selezionde dati (dal 1945-01-31 in poi)
initial_date = '1945-01-31'
df = df[df.index >= initial_date]
df.head()

,S&P 500,Earnings,10Y Usa Rate
Date,,,
1945-01-31,13.49,0.940000,2.370
1945-02-28,13.94,0.950000,2.355
1945-03-31,13.93,0.960000,2.340
1945-04-30,14.28,0.973333,2.325
1945-05-31,14.82,0.986667,2.310


Funzione ' Series to Supervised'

In [4]:
def series_to_supervised(data, n_in, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	#n_vars = 1 if type(data) is list else data.shape[1]
	#df = DataFrame(data)
 
	
	n_vars=data.shape[1] #numero di colonne del dataset
	cols, names = list(), list()

	# input sequence (t-1, ... t-n)
	for i in range(1,n_in+1):
		cols.append(data.shift(i))
		names += [(data.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
		
	# put it all together
	agg = concat(cols, axis=1) # axis=1 per affiancare orizzontalmente i dataset
	agg.columns = names
 
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True) # elimina le righe con valori NaN
	return agg

In [5]:
df_lagged_1 = series_to_supervised(df, 1)
print(df , df_lagged_1)

                S&P 500    Earnings  10Y Usa Rate
Date                                             
1945-01-31    13.490000    0.940000         2.370
1945-02-28    13.940000    0.950000         2.355
1945-03-31    13.930000    0.960000         2.340
1945-04-30    14.280000    0.973333         2.325
1945-05-31    14.820000    0.986667         2.310
...                 ...         ...           ...
2023-08-31  4457.358696  183.170000         4.170
2023-09-30  4409.095000  184.250000         4.380
2023-10-31  4269.400909  186.976667         4.800
2023-11-30  4460.063333  189.703333         4.500
2023-12-31  4685.051500  192.430000         4.020

[948 rows x 3 columns]             S&P 500(t-1)  Earnings(t-1)  10Y Usa Rate(t-1)
Date                                                      
1945-02-28     13.490000       0.940000              2.370
1945-03-31     13.940000       0.950000              2.355
1945-04-30     13.930000       0.960000              2.340
1945-05-31     14.280000       

In [6]:
df


,S&P 500,Earnings,10Y Usa Rate
Date,,,
1945-01-31,13.490000,0.940000,2.370
1945-02-28,13.940000,0.950000,2.355
1945-03-31,13.930000,0.960000,2.340
1945-04-30,14.280000,0.973333,2.325
1945-05-31,14.820000,0.986667,2.310
...,...,...,...
2023-08-31,4457.358696,183.170000,4.170
2023-09-30,4409.095000,184.250000,4.380
2023-10-31,4269.400909,186.976667,4.800


In [7]:
df_lagged_2 = series_to_supervised(df, 2)
print( df_lagged_2)

            S&P 500(t-1)  Earnings(t-1)  10Y Usa Rate(t-1)  S&P 500(t-2)  \
Date                                                                       
1945-03-31     13.940000       0.950000              2.355     13.490000   
1945-04-30     13.930000       0.960000              2.340     13.940000   
1945-05-31     14.280000       0.973333              2.325     13.930000   
1945-06-30     14.820000       0.986667              2.310     14.280000   
1945-07-31     15.090000       1.000000              2.295     14.820000   
...                  ...            ...                ...           ...   
2023-08-31   4508.075500     182.090000              3.900   4345.372857   
2023-09-30   4457.358696     183.170000              4.170   4508.075500   
2023-10-31   4409.095000     184.250000              4.380   4457.358696   
2023-11-30   4269.400909     186.976667              4.800   4409.095000   
2023-12-31   4460.063333     189.703333              4.500   4269.400909   

           

In [21]:
df_lagged_15 = series_to_supervised(df, 15)
df_lagged_15

,S&P 500(t-1),Earnings(t-1),10Y Usa Rate(t-1),S&P 500(t-2),Earnings(t-2),10Y Usa Rate(t-2),S&P 500(t-3),Earnings(t-3),10Y Usa Rate(t-3),S&P 500(t-4),...,10Y Usa Rate(t-12),S&P 500(t-13),Earnings(t-13),10Y Usa Rate(t-13),S&P 500(t-14),Earnings(t-14),10Y Usa Rate(t-14),S&P 500(t-15),Earnings(t-15),10Y Usa Rate(t-15)
Date,,,,,,,,,,,,,,,,,,,,,
1946-04-30,17.530000,0.900000,2.200,18.070000,0.920000,2.195,18.020000,0.940000,2.190,17.330000,...,2.325,13.930000,0.960000,2.340,13.940000,0.950000,2.355,13.490000,0.940000,2.370
1946-05-31,18.660000,0.880000,2.205,17.530000,0.900000,2.200,18.070000,0.920000,2.195,18.020000,...,2.310,14.280000,0.973333,2.325,13.930000,0.960000,2.340,13.940000,0.950000,2.355
1946-06-30,18.700000,0.860000,2.210,18.660000,0.880000,2.205,17.530000,0.900000,2.200,18.070000,...,2.295,14.820000,0.986667,2.310,14.280000,0.973333,2.325,13.930000,0.960000,2.340
1946-07-31,18.580000,0.840000,2.215,18.700000,0.860000,2.210,18.660000,0.880000,2.205,17.530000,...,2.280,15.090000,1.000000,2.295,14.820000,0.986667,2.310,14.280000,0.973333,2.325
1946-08-31,18.050000,0.856667,2.220,18.580000,0.840000,2.215,18.700000,0.860000,2.210,18.660000,...,2.265,14.780000,0.996667,2.280,15.090000,1.000000,2.295,14.820000,0.986667,2.310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,4508.075500,182.090000,3.900,4345.372857,181.010000,3.750,4146.173182,179.063333,3.570,4121.467368,...,2.900,3911.729500,190.583333,2.900,3898.946667,192.260000,3.140,4040.360000,194.143333,2.900
2023-09-30,4457.358696,183.170000,4.170,4508.075500,182.090000,3.900,4345.372857,181.010000,3.750,4146.173182,...,3.520,4158.563043,188.906667,2.900,3911.729500,190.583333,2.900,3898.946667,192.260000,3.140
2023-10-31,4409.095000,184.250000,4.380,4457.358696,183.170000,4.170,4508.075500,182.090000,3.900,4345.372857,...,3.980,3850.520476,187.230000,3.520,4158.563043,188.906667,2.900,3911.729500,190.583333,2.900


In [22]:
df_lagged_15.shape

(933, 45)

In [23]:
df_lagged_1.shape

(947, 3)